In [144]:
import math
import copy
fread = open("sched.txt", "r") #netdata.txt
G_v_e = fread.readline()
G_v_e = G_v_e.split('\t') #change to \t if tab spaced
V = int(G_v_e[0])
E = int(G_v_e[1].strip())
#print(E)
V_list = list(range(0,V,1))
lines = fread.readlines()
A_dict = {}
G_mat = [ [ 0 for i in range(4) ] for j in range(E) ]
g = 0
for line in lines:
    t_h = line.split('\t') #change to \t if tab spaced
    #print(t_h)
    G_mat[g][0] = int(t_h[0])
    G_mat[g][1] = int(t_h[1])
    G_mat[g][2] = int(t_h[2])
    G_mat[g][3] = float(t_h[3].replace("\n", ""))
    #print(G_mat[g][3])
    g += 1
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    if tail in A_dict.keys():
        A_dict[tail].append(head)        
    else:
        A_dict[tail] = [head]

capacity = [[0 for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    #print(i)
    #print(tail, head)
    capacity[tail][head] = G_mat[i][3]

print(G_mat, "\n")
print(A_dict)   
print(capacity)
fread.close()

[[0, 1, 0, 4.0], [0, 2, 0, 6.0], [0, 3, 0, 3.0], [0, 4, 0, 5.0], [0, 5, 0, 1.5], [1, 8, 0, 3.0], [1, 9, 0, 1.0], [1, 10, 0, 1.0], [2, 6, 0, 2.0], [2, 7, 0, 1.0], [2, 8, 0, 3.0], [3, 8, 0, 3.0], [3, 9, 0, 1.0], [3, 10, 0, 1.0], [4, 7, 0, 1.0], [4, 8, 0, 3.0], [4, 9, 0, 1.0], [4, 10, 0, 1.0], [5, 6, 0, 2.0], [5, 7, 0, 1.0], [6, 11, 0, 4.0], [7, 11, 0, 2.0], [8, 11, 0, 6.0], [9, 11, 0, 2.0], [10, 11, 0, 2.0]] 

{0: [1, 2, 3, 4, 5], 1: [8, 9, 10], 2: [6, 7, 8], 3: [8, 9, 10], 4: [7, 8, 9, 10], 5: [6, 7], 6: [11], 7: [11], 8: [11], 9: [11], 10: [11]}
[[0, 4.0, 6.0, 3.0, 5.0, 1.5, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 3.0, 1.0, 1.0, 0], [0, 0, 0, 0, 0, 0, 2.0, 1.0, 3.0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 3.0, 1.0, 1.0, 0], [0, 0, 0, 0, 0, 0, 0, 1.0, 3.0, 1.0, 1.0, 0], [0, 0, 0, 0, 0, 0, 2.0, 1.0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4.0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6.0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.0], [0, 0, 0, 0, 0, 0, 

In [145]:
def cap_Dijkstra(A_dict, cost, V, s, t):
    parent = {}
    for i in range(V):
        parent[i] = i
    dist = [math.inf]*V
    dist[s] = 0
    fixed = [s]
    chosen = s
    while (t not in fixed):
        if (chosen not in A_dict.keys()):
            print("cant use DJ here")
            return dist[t], parent
        chosen_range = A_dict[chosen]
        k = -1
        for i in range(len(chosen_range)):
            j = chosen_range[i]
            if (j in fixed):
                continue
            if (dist[j] >= dist[chosen] + cost[chosen][j]):
                dist[j] = dist[chosen] + cost[chosen][j]
                parent[j] = chosen
        min_d = 1e8
        min_v = -1
        for v in range(V):
            if ((dist[v] <= min_d) and (v not in fixed) and (dist[v] != math.inf) and (v in A_dict.keys() or v == t)):
                min_d = dist[v]
                min_v = v
        k = min_v
        if (k == -1 or k == math.inf):
            break
        else:
            fixed.append(k)
            chosen = k
    return dist[t], parent


In [146]:
def create_residual_network(G_mat, A_dict, capacity):
    R = copy.deepcopy(G_mat)
    for i in A_dict.keys():
        for j in A_dict[i]:
            if (j in A_dict.keys()):
                if (i in A_dict[j]):
                    continue
            R.append([j, i, 0, 0])
            #capacity[j][i] = 0
    return R


def find_s_t_path(R, s, end, V, capacity): # Use R for nodes & arc connects and 'capacity' for residual capacity
    mod_cost = [[math.inf for i in range(V)] for j in range(V)]
    R_dict = {}
    
    #print('capacity')
    #for i in range(V):
    #    print(capacity[i])
        
    # Develop modified cost & dictionary
    for e in range(len(R)):
        t = R[e][0]
        h = R[e][1]
        if (capacity[t][h] != 0):
            mod_cost[t][h] = 1
            if (t in R_dict.keys()):
                if (h not in R_dict[t]):
                    R_dict[t].append(h)
            else:
                R_dict[t] = [h]
        else:
            if (t in R_dict.keys()):
                if (h in R_dict[t]):
                    R_dict[t].remove(h)
        
    """
    mod_cost[3][7] = 0
    mod_cost[8][12] = 0
    mod_cost[17][18] = 0
    mod_cost[7][3] = 0
    mod_cost[12][8] = 0
    mod_cost[18][17] = 0
    """
    
    #Print values
    #print('modified cost')
    #for i in range(V):
    #    print(mod_cost[i])
    #print(R_dict)
    
    # Find shortest s-t path
    min_cost, parent = cap_Dijkstra(R_dict, mod_cost, V, s, end)
    #print(min_cost)
    #print(parent)
    path = []
    if (min_cost == math.inf):
        return path
    path = [end]
    curr = end
    while (curr != s and curr != parent[curr]):
        curr = parent[curr]
        path.insert(0, curr)
        #print(path)   
    return path

def retrieve_flow(G_mat, R, capacity):
    flow = []
    for i in range(E):
        t = G_mat[i][0]
        h = G_mat[i][1]
        old = G_mat[i][3]
        new = capacity[t][h]
        if (old > new):
            flow.append(old - new)
        else:
            flow.append(0)            
    return flow

def FF(G_mat, A_dict, capacity, V, s, end):
    R = create_residual_network(G_mat, A_dict, capacity)
    #print(R)
    
    #extra    
    """
    capacity[3][7] = 19
    capacity[8][12] = 29
    capacity[17][18] = 22
    capacity[7][3] = 0
    capacity[12][8] = 0
    capacity[18][17] = 0
    """
    

    
    
    
    flow_value = 0
    path = find_s_t_path(R, s, end, V, capacity)
    #print(path)
    while (len(path) != 0):
        min_cap = math.inf
        for n in range(len(path)-1):
            t = path[n]
            h = path[n+1]
            if (capacity[t][h] < min_cap):
                min_cap = capacity[t][h]
        #print('here')
        #print(min_cap)
        flow_value = flow_value + min_cap
        for n in range(len(path)-1):
            t = path[n]
            h = path[n+1]
            capacity[t][h] = capacity[t][h] - min_cap
            capacity[h][t] = capacity[h][t] + min_cap
        path = find_s_t_path(R, s, end, V, capacity)
        #print(path)
    flow = retrieve_flow(G_mat, R, capacity)
    #print('flow')
    #print(flow)
    return flow, flow_value
   

In [147]:
s = 0
t = 11
capacity = [[0 for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    capacity[tail][head] = G_mat[i][3]
#capacity = [[0 for i in range(V)] for j in range(V)]
    
"""
capacity[3][7] = 16
capacity[8][12] = 24
capacity[17][18] = 15
capacity[7][3] = 3
capacity[12][8] = 5
capacity[18][17] = 7
"""

flow, flow_value = FF(G_mat, A_dict, capacity, V, s, t) 
print('Max flow: ', flow_value)
#printing flow through arcs
print('Flow through arcs-')
for i in range(E):
    print('(', G_mat[i][0],',' , G_mat[i][1], ')', ':', flow[i])

Max flow:  15.5
Flow through arcs-
( 0 , 1 ) : 4.0
( 0 , 2 ) : 6.0
( 0 , 3 ) : 2.0
( 0 , 4 ) : 2.0
( 0 , 5 ) : 1.5
( 1 , 8 ) : 3.0
( 1 , 9 ) : 1.0
( 1 , 10 ) : 0
( 2 , 6 ) : 2.0
( 2 , 7 ) : 1.0
( 2 , 8 ) : 3.0
( 3 , 8 ) : 0
( 3 , 9 ) : 1.0
( 3 , 10 ) : 1.0
( 4 , 7 ) : 1.0
( 4 , 8 ) : 0
( 4 , 9 ) : 0
( 4 , 10 ) : 1.0
( 5 , 6 ) : 1.5
( 5 , 7 ) : 0
( 6 , 11 ) : 3.5
( 7 , 11 ) : 2.0
( 8 , 11 ) : 6.0
( 9 , 11 ) : 2.0
( 10 , 11 ) : 2.0


In [29]:
def FF1(G_mat, A_dict, capacity, V, s, end):
    R = create_residual_network(G_mat, A_dict, capacity)
    #print(R)
    
    #extra    
    #capacity[3][7] = 16
    #capacity[8][12] = 24
    #capacity[17][18] = 15
    
    #capacity[7][3] = 3
    #capacity[12][8] = 5
    #capacity[18][17] = 7
    
    flow_value = 0
    path = find_s_t_path(R, s, end, V, capacity)
    #print(path)
    while (len(path) != 0):
        min_cap = math.inf
        for n in range(len(path)-1):
            t = path[n]
            h = path[n+1]
            if (capacity[t][h] < min_cap):
                min_cap = capacity[t][h]
        #print('here')
        #print(min_cap)
        flow_value = flow_value + min_cap
        for n in range(len(path)-1):
            t = path[n]
            h = path[n+1]
            capacity[t][h] = capacity[t][h] - min_cap
            capacity[h][t] = capacity[h][t] + min_cap
        path = find_s_t_path(R, s, end, V, capacity)
        #print(path)
    flow = retrieve_flow(G_mat, R, capacity)
    #print('flow')
    #print(flow)
    return flow, flow_value, R

In [34]:
s = 1
t = 22
capacity = [[0 for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    capacity[tail][head] = G_mat[i][3]
    

flow, flow_value, R = FF1(G_mat, A_dict, capacity, V, s, t) 
print('Max flow: ', flow_value)
#printing flow through arcs
print('Flow through arcs-')
for i in range(E):
    print('(', G_mat[i][0], ',' , G_mat[i][1], ')', ':', flow[i])

Max flow:  27.0
Flow through arcs-
( 0 , 1 ) : 0
( 0 , 4 ) : 0
( 1 , 2 ) : 0
( 1 , 4 ) : 12.0
( 1 , 5 ) : 15.0
( 2 , 3 ) : 0
( 2 , 6 ) : 0
( 3 , 7 ) : 0
( 4 , 5 ) : 0
( 4 , 8 ) : 0
( 4 , 9 ) : 12.0
( 5 , 6 ) : 0
( 5 , 8 ) : 5.0
( 5 , 9 ) : 0
( 5 , 10 ) : 10.0
( 6 , 7 ) : 0
( 6 , 9 ) : 0
( 7 , 10 ) : 0
( 7 , 11 ) : 0
( 8 , 9 ) : 0
( 8 , 12 ) : 5.0
( 8 , 13 ) : 0
( 9 , 12 ) : 0
( 9 , 13 ) : 0
( 9 , 14 ) : 12.0
( 10 , 13 ) : 0
( 10 , 14 ) : 10.0
( 11 , 15 ) : 0
( 12 , 13 ) : 0
( 12 , 16 ) : 0
( 12 , 17 ) : 5.0
( 13 , 14 ) : 0
( 13 , 17 ) : 0
( 14 , 15 ) : 7.0
( 14 , 18 ) : 15.0
( 15 , 18 ) : 7.0
( 15 , 19 ) : 0
( 16 , 17 ) : 0
( 16 , 20 ) : 0
( 17 , 18 ) : 5.0
( 17 , 20 ) : 0
( 17 , 21 ) : 0
( 18 , 19 ) : 0
( 18 , 21 ) : 0
( 18 , 22 ) : 27.0
( 18 , 23 ) : 0
( 19 , 23 ) : 0
( 20 , 21 ) : 0
( 22 , 23 ) : 0


In [31]:
def FF2(R, G_mat, A_dict, capacity, V, s, end):
    #R = create_residual_network(G_mat, A_dict, capacity)
    #print(R)
    
    #extra    
    #capacity[3][7] = 16
    #capacity[8][12] = 24
    #capacity[17][18] = 15
    
    #capacity[7][3] = 3
    #capacity[12][8] = 5
    #capacity[18][17] = 7
    
    flow_value = 0
    path = find_s_t_path(R, s, end, V, capacity)
    #print(path)
    while (len(path) != 0):
        min_cap = math.inf
        for n in range(len(path)-1):
            t = path[n]
            h = path[n+1]
            if (capacity[t][h] < min_cap):
                min_cap = capacity[t][h]
        #print('here')
        #print(min_cap)
        flow_value = flow_value + min_cap
        for n in range(len(path)-1):
            t = path[n]
            h = path[n+1]
            capacity[t][h] = capacity[t][h] - min_cap
            capacity[h][t] = capacity[h][t] + min_cap
        path = find_s_t_path(R, s, end, V, capacity)
        #print(path)
    flow = retrieve_flow(G_mat, R, capacity)
    #print('flow')
    #print(flow)
    return flow, flow_value

In [35]:
print(capacity[3][7])
print(capacity[8][12])
print(capacity[17][18])
print(capacity[7][3])
print(capacity[12][8])
print(capacity[18][17])

19.0
24.0
17.0
0
5.0
5.0


In [37]:
s = 22
t = 1
#capacity[3][7] = 16
#capacity[8][12] = 24
#capacity[17][18] = 15
capacity[7][3] = 3
capacity[12][8] = 5
capacity[18][17] = 7
flow, flow_value = FF2(R, G_mat, A_dict, capacity, V, s, t) 
print('Max flow: ', flow_value)
#printing flow through arcs
print('Flow through arcs-')
for i in range(E):
    print('(', G_mat[i][0], ',' , G_mat[i][1], ')', ':', flow[i])

Max flow:  0
Flow through arcs-
( 0 , 1 ) : 0
( 0 , 4 ) : 0
( 1 , 2 ) : 0
( 1 , 4 ) : 0
( 1 , 5 ) : 0
( 2 , 3 ) : 0
( 2 , 6 ) : 0
( 3 , 7 ) : 0
( 4 , 5 ) : 0
( 4 , 8 ) : 0
( 4 , 9 ) : 0
( 5 , 6 ) : 0
( 5 , 8 ) : 0
( 5 , 9 ) : 0
( 5 , 10 ) : 0
( 6 , 7 ) : 0
( 6 , 9 ) : 0
( 7 , 10 ) : 0
( 7 , 11 ) : 0
( 8 , 9 ) : 0
( 8 , 12 ) : 0
( 8 , 13 ) : 0
( 9 , 12 ) : 0
( 9 , 13 ) : 0
( 9 , 14 ) : 0
( 10 , 13 ) : 0
( 10 , 14 ) : 0
( 11 , 15 ) : 0
( 12 , 13 ) : 0
( 12 , 16 ) : 0
( 12 , 17 ) : 0
( 13 , 14 ) : 0
( 13 , 17 ) : 0
( 14 , 15 ) : 0
( 14 , 18 ) : 0
( 15 , 18 ) : 0
( 15 , 19 ) : 0
( 16 , 17 ) : 0
( 16 , 20 ) : 0
( 17 , 18 ) : 0
( 17 , 20 ) : 0
( 17 , 21 ) : 0
( 18 , 19 ) : 0
( 18 , 21 ) : 0
( 18 , 22 ) : 0
( 18 , 23 ) : 0
( 19 , 23 ) : 0
( 20 , 21 ) : 0
( 22 , 23 ) : 0
